In [15]:
import pprint
from gensim.models import Word2Vec

import nltk
from nltk.corpus import stopwords
from bs4 import BeautifulSoup

list_of_dicts = []
dict_of_data = {}
# Opens a handle to the file, but does not read anything yet.
fh = open('movies.small.txt')

# This is more memory efficient.
# Reads one line at a time.
for line in fh.readlines():
    # Strip off leading and trailing white-space.
    line = line.strip()
    if line != '':
        # Assume it's safe to split:
        label, data = line.split(': ', 1)
        shortkey = label.split('/')[1]
        dict_of_data[shortkey] = data
    else:
        # We're at the end of a record. Everything we want is in the
        # dictionary.
        list_of_dicts.append(dict_of_data)
        dict_of_data = {}

# Look at the first three items.
pprint.pprint((list_of_dicts[0:3]))

[{'helpfulness': '7/7',
  'productId': 'B003AI2VGA',
  'profileName': 'Brian E. Erland "Rainbow Sphinx"',
  'score': '3.0',
  'summary': '"There Is So Much Darkness Now ~ Come For The Miracle"',
  'text': 'Synopsis: On the daily trek from Juarez, Mexico to El Paso, Texas an ever increasing number of female workers are found raped and murdered in the surrounding desert. Investigative reporter Karina Danes (Minnie Driver) arrives from Los Angeles to pursue the story and angers both the local police and the factory owners who employee the undocumented aliens with her pointed questions and relentless quest for the truth.<br /><br />Her story goes nationwide when a young girl named Mariela (Ana Claudia Talancon) survives a vicious attack and walks out of the desert crediting the Blessed Virgin for her rescue. Her story is further enhanced when the "Wounds of Christ" (stigmata) appear in her palms. She also claims to have received a message of hope for the Virgin Mary and soon a fanatical mo

In [16]:
import re
def text_to_wordlist(text, remove_stopwords=False):
    text = re.sub("[^a-zA-Z]", " ", text)
    words = text.lower().split()
    # Optionally remove stop words (false by default)
    if remove_stopwords:
        stops = set(stopwords.words("english"))
        words = [w for w in words if not w in stops]
    # 5. Return a list of words
    return words


def make_feature_matrix(words, model):
    # feature_matrix = np.zeros((len(words), num_features), dtype="float32")
    feature_matrix = []
    # counter = 0.
    for word in words:
        if word in model.vocab:
            feature_matrix.append(list(model[word]))
            # feature_matrix[counter] = model[word]
            # counter += 1
        # else:
        #    print 'word', word, 'is not in a model\n'
    feature_matrix = np.array(feature_matrix)
    return feature_matrix

In [17]:
###### Modify in place.
import datetime
for review in list_of_dicts:
    # Convert to float
    review['score'] = float(review['score'])
    # Convert to readable time
    int_timestamp = int(review['time'])
    datetime_obj = datetime.datetime.fromtimestamp(int_timestamp)
    str_iso_datetime = datetime_obj.isoformat()
    review['time'] = str_iso_datetime
    
    review['text'] = text_to_wordlist(BeautifulSoup(review['text']).get_text(), nltk.data.load('tokenizers/punkt/english.pickle'))

# Modify in place some more!!
# Convert helpfulness to a float.
for review in list_of_dicts:
    helpful_str, numberof_str = review['helpfulness'].split('/')
    helpful_int = int(helpful_str)
    numberof_int = int(numberof_str)
    if numberof_int == 0: # Why is this needed?
        numberof_int = 1.0
    review['helpfulness'] = float(helpful_int)/numberof_int
    review['helpful'] = helpful_int
    review['total'] = numberof_int

In [23]:
sentences = []
print("Parsing sentences from training set")
for review in (list_of_dicts):
    sentences += review['text']
print len(sentences)
print sentences[0]
print sentences[1]
print len(list_of_dicts)

Parsing sentences from training set
474550
synopsis
daily
5554


In [24]:
wordfreq = {}
for word in sentences:
    if word not in wordfreq:
        wordfreq[word] = 0 
    wordfreq[word] += 1

In [25]:
from gensim.models import Word2Vec
import datetime

import nltk
from nltk.corpus import stopwords

num_features = 100    # Word vector dimensionality
min_word_count = 2   # Minimum word count
num_workers = 2       # Number of threads to run in parallel
context = 10          # Context window size
downsampling = 1e-3   # Downsample setting for frequent words

print("Training model...")
model = Word2Vec(sentences, workers=num_workers, \
            size=num_features, min_count = min_word_count, \
            window = context, sample = downsampling)
model_name = "100features_40minwords_10context" + datetime.datetime.now().strftime('%Y%m%d%H%M%S')
model.save(model_name)

from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer(analyzer = 'word', tokenizer = None, preprocessor = None, stop_words = None, max_features = 100)

words_features = vectorizer.fit_transform(sentences)

words_features = words_features.toarray()

print words_features.shape

Training model...
(474550, 100)


In [31]:
import operator
sorted_x = sorted(wordfreq.items(), key=operator.itemgetter(1))[::-1][:100]
sorts = map(lambda x:x[0], sorted_x)
print sorts

[u'movie', u'film', u'one', u'like', u'good', u'great', u'story', u'time', u'would', u'well', u'see', u'much', u'really', u'dvd', u'even', u'also', u'first', u'get', u'movies', u'samurai', u'love', u'best', u'way', u'many', u'people', u'watch', u'character', u'could', u'life', u'think', u'little', u'still', u'better', u'make', u'know', u'made', u'films', u'scenes', u'two', u'never', u'action', u'back', u'characters', u'man', u'new', u'end', u'kids', u'seen', u'years', u'say', u'last', u'acting', u'watching', u'john', u'old', u'terminator', u'us', u'cruise', u'series', u'scene', u'ever', u'book', u'real', u'plot', u'tom', u'go', u'young', u'nash', u'though', u'want', u'director', u'something', u'bad', u'beautiful', u'mind', u'part', u'show', u'makes', u'find', u'another', u'special', u'every', u'however', u'japanese', u'long', u'excellent', u'world', u'original', u'version', u'lot', u'actually', u'actors', u'must', u'times', u'performance', u'things', u'done', u'going', u'set', u'feel']

In [32]:
words = {}
for i, e in enumerate(sorts):
    words[e] = i

In [43]:
import numpy as np
Y = 5 * np.zeros((100,156))

In [47]:
movieses = {}
for review in list_of_dicts:
    id = review['productId']
    if id not in movieses:
        movieses[id] = []
    movieses[id] += review['text']
mov = movieses.keys()

In [49]:
import pandas as pd
df = pd.DataFrame(list_of_dicts)

In [51]:
df.head()

,helpful,helpfulness,productId,profileName,score,summary,text,time,total,userId
0,7,1.0,B003AI2VGA,"Brian E. Erland ""Rainbow Sphinx""",3.0,"""There Is So Much Darkness Now ~ Come For The ...","[synopsis, daily, trek, juarez, mexico, el, pa...",2007-06-25T04:00:00,7.0,A141HP4LYPWMSR
1,4,1.0,B003AI2VGA,Grady Harp,3.0,Worthwhile and Important Story Hampered by Poo...,"[virgin, juarez, based, true, events, surround...",2007-06-16T04:00:00,4.0,A328S9RN3U5M68
2,8,0.8,B003AI2VGA,"Chrissy K. McVay ""Writer""",5.0,This movie needed to be made.,"[scenes, film, disquieting, due, graphic, enac...",2006-11-30T03:00:00,10.0,A1I7QGUDP043DG
3,1,1.0,B003AI2VGA,golgotha.gov,3.0,distantly based on a real tragedy,"[virgin, juarez, directed, kevin, james, dobso...",2007-12-09T03:00:00,1.0,A1M5405JH9THP9
4,1,1.0,B003AI2VGA,"KerrLines ""&#34;Movies,Music,Theatre&#34;""",3.0,"""What's going on down in Juarez and shining a ...","[informationally, showtime, original, essentia...",2007-08-29T04:00:00,1.0,ATXL536YX71TR


In [52]:
rp = df.pivot_table(columns=['productId'], index=['userId'],values='score')
rp.head()

productId,0790747324,0800103688,1562229567,1573470937,1888617047,6300147967,6300157423,6300213153,6300246299,6301173147,...,B007EMEOEA,B007NF02Q2,B0085KGGLY,B0088QNUUK,B008BSM1EG,B008FPU7AA,B0092MDYV4,B0095D5454,B0099VUXCQ,B009B6XA1U
userId,,,,,,,,,,,,,,,,,,,,,
A100Y8WSLFJN7Q,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
A103EXN5Q7HX6Z,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN
A103VZ3KDF2RT5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
A103ZG6ASSR7UT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
A1041HQGJDKFG5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [81]:
productId = list(rp.columns.values)

In [53]:
rp = rp.fillna(0); # Replace NaN
rp.head()

productId,0790747324,0800103688,1562229567,1573470937,1888617047,6300147967,6300157423,6300213153,6300246299,6301173147,...,B007EMEOEA,B007NF02Q2,B0085KGGLY,B0088QNUUK,B008BSM1EG,B008FPU7AA,B0092MDYV4,B0095D5454,B0099VUXCQ,B009B6XA1U
userId,,,,,,,,,,,,,,,,,,,,,
A100Y8WSLFJN7Q,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
A103EXN5Q7HX6Z,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
A103VZ3KDF2RT5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
A103ZG6ASSR7UT,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
A1041HQGJDKFG5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [54]:
Q = rp.values

In [55]:
Q.shape

(5113, 156)

In [113]:
lambda_ = 0.1
n_factors = 100
m, n = Q.shape
n_iterations = 101

In [134]:
X = 5 * np.random.rand(m, n_factors) 
Y = 5 * np.random.rand(n_factors, n)
def get_error(Q, X, Y, W):
    return np.sum((W * (Q - np.dot(X, Y)))**2)


In [135]:
Y.shape

(100, 156)

In [136]:
for i,movie in enumerate(mov):
    a = np.zeros(100)
    a_word = movieses.get(movie)
    for word in a_word:
        if word in words:
            a[words[word]] += 1
    #if a.sum() != 0:
    Y[:,productId.index(movie)] = a

In [137]:
W = Q>0.5
W[W == True] = 1
W[W == False] = 0
# To be consistent with our Q matrix
W = W.astype(np.float64, copy=False)

In [138]:
errors = []
import time
start_time = time.time()
for ii in xrange(n_iterations):
    X = np.linalg.solve(np.dot(Y, Y.T) + lambda_ * np.eye(n_factors), 
                        np.dot(Y, Q.T)).T
    Y = np.linalg.solve(np.dot(X.T, X) + lambda_ * np.eye(n_factors),
                        np.dot(X.T, Q))
    errors.append(get_error(Q, X, Y, W))
    if ii % 100 == 0:
        print('{}th iteration is completed'.format(ii)), errors[len(errors)-1]
Q_hat = np.dot(X, Y)
print 'time is ', time.time() -  start_time
print('Error of rated movies: {}'.format(get_error(Q, X, Y, W)))

0th iteration is completed 38149.3314151
100th iteration is completed 883.95263481
200th iteration is completed 882.552156189
300th iteration is completed 882.289980104
400th iteration is completed 882.20393845
500th iteration is completed 882.16777758
600th iteration is completed 882.15028473
700th iteration is completed 882.141034548
800th iteration is completed 882.135840538
900th iteration is completed 882.132797203
1000th iteration is completed 882.130956691
time is  58.4737370014
Error of rated movies: 882.130956691


In [139]:
X = 5 * np.random.rand(m, n_factors) 
Y = 5 * np.random.rand(n_factors, n)
def get_error(Q, X, Y, W):
    return np.sum((W * (Q - np.dot(X, Y)))**2)


In [140]:
errors = []
import time
start_time = time.time()
for ii in xrange(n_iterations):
    X = np.linalg.solve(np.dot(Y, Y.T) + lambda_ * np.eye(n_factors), 
                        np.dot(Y, Q.T)).T
    Y = np.linalg.solve(np.dot(X.T, X) + lambda_ * np.eye(n_factors),
                        np.dot(X.T, Q))
    errors.append(get_error(Q, X, Y, W))
    if ii % 100 == 0:
        print('{}th iteration is completed'.format(ii)), errors[len(errors)-1]
Q_hat = np.dot(X, Y)
print 'time is ', time.time() -  start_time
print('Error of rated movies: {}'.format(get_error(Q, X, Y, W)))

0th iteration is completed 2119.92035832
100th iteration is completed 882.838888757
200th iteration is completed 882.308752096
300th iteration is completed 882.198392203
400th iteration is completed 882.160850441
500th iteration is completed 882.144929636
600th iteration is completed 882.137249123
700th iteration is completed 882.133226316
800th iteration is completed 882.13100009
900th iteration is completed 882.129719565
1000th iteration is completed 882.128961694
time is  62.7628910542
Error of rated movies: 882.128961694
